# 3. Regression

In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

## Regression intro

- What is regression
- Supervised learning

### Linear regression

- Linear model
- Common approaches
  - Least squares
  - Bayesian linear regression

### California housing dataset

The California housing dataset includes

In [30]:
housing_data = fetch_california_housing(as_frame=True)

df = housing_data['frame']
print(df.columns)

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup',
       'Latitude', 'Longitude', 'MedHouseVal'],
      dtype='object')


The housing data comprises of 8 features:

- `MedInc`: Median income of the population in the block.
- `HouseAge`: Average age of houses in the block.
- `AveRooms`: Average number of rooms in a house in the block.
- `AveBedrms`: Average number of bedrooms in a house in the block.
- `Population`: Population of the block.
- `AveOccup`: The average number of people living in a house.
- `Latitude`: Latitudinal position.
- `Logitude`: Longitudinal position.
- `MedHouseVal`: The median value of houses in the block.
 
The target feature is `MedHouseVal`.

We shall now look at the datatypes for each feature.

In [ ]:
print(df.dtypes)

Each of the features are numerical, as is the target feature which makes regression easier.

### Geographical data

As the data is geographical it is a good idea to visualise it as such.

Using the `Latitude` and `Longitude` features we can plot the data as a scatter plot. Using the `alpha` parameter we can get an idea of population density across California.

In [ ]:
latitude = df['Latitude']
longitude = df['Longitude']

fig, ax = plt.subplots(figsize=(12, 8))
plt.title("Geographical Plot of California Housing Data")
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.scatter(latitude, longitude, s=10, alpha=0.25)
plt.show()

We can assume the whitespace in the lower left corner is due to no houses being located in the ocean. This means the high density points are along the coast, which may have a high correlation with house price.

In [ ]:
house_val = df['MedHouseVal']
population = df['Population']

fig, ax = plt.subplots(figsize=(12, 8))
plt.title("California Housing Value and Geographical Location")
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.scatter(latitude, longitude, s=population/100, alpha=0.5, c=house_val, cmap='coolwarm')
plt.colorbar().set_label("Median House Value (scale of 0-5)")
plt.show()

From the scatter plot above we can see the more expensive (red) areas are closely packed, with the cheaper (blue) areas being far more scattered, as well as (generally) being further away from the coast. We can assume the more expensive areas relate to Los Angeles (north) and San Fransisco (south). We will need to ensure geographical data is encoded into the model as location evidetely has an impact on house price.

### Correlation and distribution

To get a quick look at the characteristics of the dataset we can use the `describe()` function.

In [ ]:
df.describe()

From this we can see that some of the features have some large outlying values such as `AveRooms` and `AveOccup`, due to the max value being so much higher than the 75th percentile. We will have to handle this if we want to look at the histogram of the data as it may skew the results and not let us see the real distribution.

As we have already seen geographical data has a large impact on house value we will not explore its correlation with the other features any further. We will find the correlation between each of the remaining features.

In [ ]:
non_geo = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms',
           'Population', 'AveOccup', 'MedHouseVal']

corr = df[non_geo].corr()
print(corr['MedHouseVal'].abs().sort_values(ascending=False))

We can see some inital results, such as income being highly correlated with house value but it will be easier to visualise as a correlation matrix.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.set_title("Correlation of Non-Geographical Features", fontsize=16)
mat = ax.matshow(corr)
ax.set_xticks(range(len(non_geo)))
ax.set_yticks(range(len(non_geo)))
ax.set_xticklabels(non_geo, rotation=-90, fontsize=12)
ax.set_yticklabels(non_geo, fontsize=12)
plt.colorbar(mat, ax=ax)
plt.show()

We shall remove the highest values from some of the features with a long tail so we can get a better look at the disribution.

There are also some anomalies with the median house value and house age features. It seems that they have cut-offs of 5 and 50 repsectively and so there are a high number of them in the final bin on the histogram. This will have to be dealt with as to not skew the model.

In [ ]:
df_nohigh = df.astype(float)
df_nohigh = df_nohigh.drop(df_nohigh[(df_nohigh['MedInc'] > 10) |
                                     (df_nohigh['HouseAge'] > 50) |
                                     (df_nohigh['AveRooms'] > 10) |
                                     (df_nohigh['AveBedrms'] > 2) |
                                     (df_nohigh['Population'] > 6000) |
                                     (df_nohigh['AveOccup'] > 8) |
                                     (df_nohigh['MedHouseVal'] > 4.7)].index)

In [ ]:
df_nohigh[non_geo].hist(bins=50, figsize=(12, 8))

It appears most features have a lognormal distribution - that is they rise sharply with a long right-hand tail. This is due to a a small number of blocks having abnormally large, high value homes.

Taking the log of these features should mean they are better fit by a normal distribution.

In [ ]:
df_nohighnormal = df_nohigh.copy()
df_nohighnormal["MedInc"] = np.log1p(df_nohighnormal['MedInc'])
df_nohighnormal["AveRooms"] = np.log1p(df_nohighnormal['AveRooms'])
df_nohighnormal["AveBedrms"] = np.log1p(df_nohighnormal['AveBedrms'])
df_nohighnormal["Population"] = np.log1p(df_nohighnormal['Population'])
df_nohighnormal["AveOccup"] = np.log1p(df_nohighnormal['AveOccup'])
df_nohighnormal["MedHouseVal"] = np.log1p(df_nohighnormal['MedHouseVal'])

df_nohighnormal[non_geo].hist(bins=50, figsize=(12, 8))

We shall apply this transformation to our dataframe .

In [ ]:
df_normal = df.copy()
df_normal["MedInc"] = np.log1p(df_normal['MedInc'])
df_normal["AveRooms"] = np.log1p(df_normal['AveRooms'])
df_normal["AveBedrms"] = np.log1p(df_normal['AveBedrms'])
df_normal["Population"] = np.log1p(df_normal['Population'])
df_normal["AveOccup"] = np.log1p(df_normal['AveOccup'])
df_normal["MedHouseVal"] = np.log1p(df_normal['MedHouseVal'])

### Missing data

Missing data will cause issues when it comes to training a model, so we will need to check for and deal with any missing data points.

In [ ]:
df_normal.isnull().sum()

This seems to be a very clean dataset and does not have any missing values.

### Outliers

As with missing values, any outliers in the data could also cause our model to underperform.

In [ ]:
df_cut = df_normal.copy()
df_cut[non_geo].hist(bins=50, figsize=(16, 16))

It seems that the features `MedHouseVal` and `HouseAge` have been capped, this means there aren't any outliers but a large number of maximum values.

Had these values not been capped the data would have followed more of a lognormal distribution. That is the right-hand tail would have stretched out further slowly decreasing.

There is no way of recovering the lost data so instead we call cut off the cut-off. Later we shall scale the data so it should flatten out and be more normally distributed.

In [ ]:
# df_nohigh = df[non_geo].astype(float)
df_cut = df_cut.drop(df_cut[(df_cut['HouseAge'] > 50) |
                            (df_cut['MedHouseVal'] > 1.75)].index)

df_cut.hist(bins=50, figsize=(12, 8))

### Feature engineering

It can be benificial to create additional features.

In [ ]:
print(df_cut.columns)

In [ ]:
extra_df = df_cut.copy()

extra_df['AveBedrmsPerRoom'] = extra_df['AveBedrms'] / extra_df['AveRooms']
extra_df['AveBedrmsPerOccup'] = extra_df['AveBedrms'] / extra_df['AveOccup']
extra_df['AveAddRooms'] = extra_df['AveRooms'] - extra_df['AveBedrms']
extra_df['EstHouses'] = extra_df['Population'] / extra_df['AveOccup']

new_features = ['MedHouseVal', 'AveBedrmsPerRoom', 'AveBedrmsPerOccup',
                'AveAddRooms', 'EstHouses']

new_features_corr = extra_df.corr()
new_features_corr['MedHouseVal'].abs().sort_values(ascending=False)

# fig, ax = plt.subplots(figsize=(12, 8))
# ax.set_title("Correlation of Non-Geographical Features", fontsize=16)
# mat = ax.matshow(new_features_corr)
# ax.set_xticks(range(len(new_features)))
# ax.set_yticks(range(len(new_features)))
# ax.set_xticklabels(new_features, rotation=-90, fontsize=12)
# ax.set_yticklabels(new_features, fontsize=12)
# plt.colorbar(mat, ax=ax)
# plt.show()

It seems the new bedrooms per room and additional rooms features are both better features than the individual bedroom and room features. We shall keep these and remove the original features. Estimated houses also seems to have added some good information so we shall keep it in the feature set.

The average bedrooms per person has not added any more information to the dataset so we shall remove this.

As for the geographical data we shall encode this later on.

#### Creating geographical features

As mentioned above, the geographical features `Latitude` and `Longitude` can be transformed into a single feature to represent distance to the closest city (LA and San Fransisco). The longitude and latitude seem to line up with real life positions so we shall get the positions of LA and San Fransisco from Google maps.

In [ ]:
# LA location as north-westerly peak
la_lat, la_lon = 34.0522, -118.2437
# SF location as south-easterly peak
sf_lat, sf_lon = 37.7749, -122.4194

extra_df['DistToLA'] = np.sqrt((extra_df['Latitude'] - la_lat)**2 +
                               (extra_df['Longitude'] - la_lon)**2)
extra_df['DistToSF'] = np.sqrt((extra_df['Latitude'] - sf_lat)**2 +
                               (extra_df['Longitude'] - sf_lon)**2)

extra_df['DistToCity'] = extra_df[['DistToLA', 'DistToSF']].min(axis=1)

Now lets look at the correlation of this new feature with median house price.

In [ ]:
city_corr = extra_df.corr()
city_corr['MedHouseVal'].abs().sort_values(ascending=False)

This has created another great feature. As the distance to city feature is so much stronger than any other geographical feature we shall use this and drop the others.

We shall also check the distribution of this new feature.

In [ ]:
extra_df['DistToCity'].hist(bins=50, figsize=(12, 8))

The distribution doesnt seem clear which is likely down to the very centre of the city not being the most expensive and decreasing as they go out.

To add some more accuracy we can add some more known expensive areas around California. A quick google search lists the following cities as the top 10 most expensive areas as:

- Atherton (37.4613° N 122.1997° W)
- Santa Monica (34.0195° N, 118.4912° W)
- Beverly Hills (34.0736° N, 118.4004° W)
- Palo Alto (37.4419° N, 122.1430° W)
- Los Altos (94022) (37.3852° N, 122.1141°W)
- Ross (37.9624° N, 122.5550° W)
- Portola Valley (37.3841° N, 122.2352° W)
- Los Altos (94024) (37.3478° N, 122.1008° W)
- Newport Beach (92661) (33.6189° N, 117.9298° W)
- Newport Beach (92662) (33.6061° N, 117.8912° W)

I shall also add a point for San Diego at 32.7157° N, 117.1611° W.

We shall calculate the distances for each bloc to these areas and add a new feature to represent the miniumum of these.

In [ ]:
town_coords = np.array([[37.4613, -122.1997],
                        [34.0195, -118.4912],
                        [34.0736, -118.4004],
                        [37.4419, -122.1430],
                        [37.3852, -122.1141],
                        [37.9624, -122.5550],
                        [37.3841, -122.2352],
                        [37.3478, -122.1008],
                        [33.6189, -117.9298],
                        [33.6061, -117.8912],
                        [32.7157, -117.1611]])

coords = np.asarray(extra_df[['Latitude', 'Longitude']])

town_dists = np.asarray([np.sqrt((coords[:, 0] - town_coord[0])**2 + 
                                 (coords[:, 1] - town_coord[1])**2) 
                         for town_coord in town_coords])

dist_to_town = np.min(town_dists, axis=0)

We can now add the new feature and compare it to the original distance to city.

In [ ]:
extra_df['DistToTown'] = dist_to_town
extra_df['DistToTown'].hist(bins=50, figsize=(12, 8))

In [ ]:
city_corr = extra_df.corr()
city_corr['MedHouseVal'].abs().sort_values(ascending=False)

As I hoped this had improved the correlation further.

We can now transform this to a normal distribution.

In [ ]:
qt = QuantileTransformer(output_distribution="normal")
normal_dist_to_town = qt.fit_transform(dist_to_town.reshape(-1, 1))

extra_df['DistToTown'] = normal_dist_to_town
extra_df['DistToTown'].hist(bins=100, figsize=(12, 8))

### Categorical features

Categorical features in the data usually need to be encoded into numerical features for a machine learning algorithm to learn from them, especially with a regression model.

With this dataset there are only numerical features and all are of float type, so we can leave them as they are.

### Feature scaling

Now we have created all the features we shall used we shall apply scaling so features are centered with 0 mean.

In [ ]:
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(extra_df),
                         columns=extra_df.columns)

df_scaled.hist(figsize=(16, 16), bins=50)

Another scaling technique we can try is sklearns quantile transformer. It attempts to fit the data to a normal distribution which should help us when it comes to picking our Bayesian prior distribution.

In [ ]:
qt = QuantileTransformer(output_distribution="normal")
normal_transformed_df = pd.DataFrame(qt.fit_transform(extra_df),
                                     columns=extra_df.columns)

normal_transformed_df.hist(figsize=(16, 16), bins=50)

This has worked well and we can clearly see that the new transformation has normally distributed most of the features.

One notable outlier is the house age feature. My hypothesis is that house age cannot be normally distributed as both old and new houses can be expensive. It does not seem to be a feature that correlates well with house price so we may drop it later on when it comes to selecting the final features.

### Feature Selection

As discussed in part one reducing the number of dimensions can also be benificial for both the training time and generalisation of the model. This can be done by removing unnescessary features, but also by combining similar features.

In [ ]:
corr = normal_transformed_df.corr()
print(corr["MedHouseVal"].abs().sort_values(ascending=False))

In [ ]:
sns.pairplot(normal_transformed_df[non_geo])

We shall remove the bedroom and room features as the new bedrooms per room feature has better correlation.

As discussed earlier we shall remove all of the geographical features apart from `DistToTown`.

We shall also remove the bedrooms per person feature we created as it does not provide any more useful information.

The population and house age features also seem to have little correlation to house price also so we shall also remove those.

In [ ]:
normal_transformed_df.columns

In [ ]:
features_to_remove = ['AveRooms', 'AveBedrms', 'Latitude', 'Longitude',
                      'DistToLA', 'DistToSF', 'DistToCity',
                      'AveBedrmsPerOccup', 'Population', 'HouseAge']

data = normal_transformed_df.drop(features_to_remove, axis=1, inplace=False)
data.head()

In [ ]:
data.corr()['MedHouseVal'].abs().sort_values(ascending=False)

In [ ]:
sns.pairplot(data)

We now have the most correlating features, whilst not duplicating any information between them.

We have also reduced the dimensionality of the data which should aid both model training ease and generalisation.

### Notes on sampling

- In group bias
- Ground truth
- Prior belief
- Sampling bias
- Selection bias
- Bias variance trade-off

## Bayesian Linear Regression

#### Bayes Theorem

- Bayes theorem
- Bayes rule
- Bayesian statistics

#### MCMC

- Markov chain
- Markov property
- MCMC
- Metropolis Hastings

#### Bayesian linear regression

- What is bayesian linear regression

#### Prior beliefs

- Prior for Bayesian
- Log-normal distribution

#### Prepare data for model

- Test train val split
- Scaling

In [ ]:
print(data.columns)

#### Spilt features and labels

Seperate features and labels and create new dataframes.

In [ ]:
X_features = ['MedInc', 'AveOccup', 'AveBedrmsPerRoom',
              'AveAddRooms', 'EstHouses', 'DistToTown']
y_features = ['MedHouseVal']

X = data[X_features]
y = data[y_features]

#### Get test and train sets

Create testing and training sets. Combine features and labels into train and test data frames.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
data_train = pd.concat((y_train, X_train), axis=1)
data_test = pd.concat((y_test, X_test), axis=1)

#### Data processing pipeline

For convenience I have created a module which provides the above data pipeline.

It will return the california housing data as 4 Pandas DataFrames:

- X_train - training data
- X_test - testing data
- y_train - real house prices for training
- y_test - real house prices for testing

In [4]:
import california_data_pipeline as cp
import importlib
importlib.reload(cp)

X_train, X_test, y_train, y_test = cp.load_train_test()

In [33]:
X_train.describe()

,MedInc,AveOccup,AveBedrmsPerRoom,AveAddRooms,EstHouses,DistToTown
count,14735.000000,14735.000000,14735.000000,14735.000000,14735.000000,14735.000000
mean,-0.002548,0.006250,-0.000037,-0.001734,-0.003692,0.004260
std,1.004026,1.001917,0.999662,0.999165,1.002058,0.997231
min,-5.199338,-3.201766,-5.199338,-5.199338,-5.199338,-5.199338
25%,-0.676394,-0.670385,-0.674644,-0.677818,-0.684667,-0.666649
50%,0.003764,0.002503,-0.004090,-0.001048,-0.006362,0.002993
75%,0.673733,0.686719,0.676533,0.672609,0.678650,0.672975
max,5.199338,5.199338,5.199338,5.199338,5.199338,5.199338


In [34]:
y_train.describe()

,MedHouseVal
count,14735.000000
mean,-0.003381
std,1.000829
min,-5.199338
25%,-0.670557
50%,-0.001255
75%,0.667873
max,5.199338


#### Performance measure

- Define the performance measure of the model.
- MSE / RMSE

#### Create the model

Create initial Bayesian linear regression model with guassian priors.

In [ ]:
# def normal_linreg(data, formula):
#     with pm.Model() as model:
#         # Normal prior
#         family = pm.glm.families.Normal()
#         # Create model from formula
#         pm.GLM.from_formula(formula, data=data, family=family)
#         return model

#### Model sampler

Method to sample model and return trace.

In [ ]:
# def sample_model(n_samples=1000, n_tune=500, n_cores=2):
#     trace = pm.sample(draws=n_samples, tune=n_tune,
#                       progressbar=True, cores=n_cores)
#     return trace

### Sample the model

In [ ]:
# formula = "MedHouseVal ~ MedInc + AveOccup + AveBedrmsPerRoom +\
#            AveAddRooms + EstHouses + DistToCity"

# with normal_linreg(data_train, formula):
#     normal_linreg_trace = sample_model(n_samples=5000, n_tune=500,
#                                        n_cores=16)

In [ ]:
# pm.save_trace(normal_linreg_trace, directory="traces", overwrite=True)

#### Evaluation of model

- Evaluate model performance on val / test set

#### Analyse the sample trace

In [ ]:
# pm.traceplot(normal_linreg_trace)

In [ ]:
# pm.plot_posterior(normal_linreg_trace)

In [ ]:
# pm.forestplot(normal_linreg_trace)

#### Test the model

In [ ]:
# # calculate mae and rmse for model on X, y
# def score(trace, X, y):
#     # get mean of coefficients
#     all_coeffs = np.asarray([trace[name] for name in trace.varnames])
#     coeff_means = all_coeffs.mean(axis=1)
#     # set intercept and variable coefficients
#     n_coeffs = X.shape[1]
#     coeffs = coeff_means[:n_coeffs+1]
#     # get predictions
#     X = np.column_stack((np.ones((X.shape[0])), X))
#     preds = np.dot(coeffs, X.T)
#     # calculate mae, rmse
#     labels = np.asarray(y).reshape(-1)
#     errors = preds - labels
#     mae = np.mean(abs(errors))
#     rmse = np.sqrt(np.mean(errors**2))
#     return (mae, rmse)

In [ ]:
# mae, rmse = score(normal_linreg_trace, X_test, y_test)

# print("Bayesian Linear Regression Mean Results")
# print(f"Mean absolute error     (MAE)  : {mae:.3f}")
# print(f"Root mean squared error (RMSE) : {rmse:.3f}")

### Compare to standard linear regression

In [5]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

In [19]:
linreg_preds = linreg.predict(X_test)
linreg_r2 = r2_score(y_test, linreg_preds)
linreg_mse = mean_squared_error(y_test, linreg_preds)
linreg_rmse = mean_squared_error(y_test, linreg_preds, squared=False)
print("Linear regression R2:", linreg_r2)
print("Linear regression MSE:", linreg_mse)
print("Linear regression RMSE", linreg_rmse)

Linear regression R2: 0.6244136481772364
Linear regression MSE: 0.3728552495507357
Linear regression RMSE 0.6106187432029381


### Scikit-learn wrapper

I have created a scikit-learn wrapper class for the model above which gives the model sklearn functionality.

Create an instance of `BayesianLinearRegression`:

In [2]:
import bayesian_linear_regressor as br
import importlib
importlib.reload(br)

bayesianreg = br.BayesianLinearRegression()

Call `fit()` with the training features `X` and real values `y` with the model formula create and sample from the model.

Additional parameters `n_samples`, `n_tune` and `n_cores` are available.

In [14]:
bayesianreg.fit(X_train, y_train, n_samples=5000, n_tune=2000, n_cores=8)

Sampling with formula:
MedHouseVal ~ MedInc + AveOccup + AveBedrmsPerRoom + AveAddRooms + EstHouses + DistToTown



Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (8 chains in 8 jobs)
NUTS: [sd, DistToTown, EstHouses, AveAddRooms, AveBedrmsPerRoom, AveOccup, MedInc, Intercept]
Sampling 8 chains, 0 divergences: 100%|██████████| 56000/56000 [00:44<00:00, 1255.00draws/s]


BayesianLinearRegression()

This will also instanciate the objects model and trace attributes:`bayesianreg.model_` and RMSE: `bayesianreg.trace_`.

Call `predict` with test features `X` to return predicted values.

In [15]:
preds = bayesianreg.predict(X_test)

This will also instanciate the objects mean coefficients attribute:`bayesianreg.mean_coeffs_`.

In [16]:
print(bayesianreg.mean_coeffs_)

[ 0.00173039  0.62151252 -0.14779621  0.09071183  0.02570703  0.0339327
 -0.3963465  -0.50286574  0.6048055 ]


Call `score()` with test features `X` and real values `y` to return the R2 score of the model on the training data.

This will also instanciate the objects attributes for MAE:`bayesianreg.mae_` and RMSE: `bayesianreg.rmse_`.

In [20]:
r2 = bayesianreg.score(X_test, y_test)
print(f"Bayesian linear regression R2: {r2}")
print(f"Bayesian linear regression MSE:  {bayesianreg.mse_}")
print(f"Bayesian linear regression RMSE: {bayesianreg.rmse_}")

Bayesian linear regression R2: 0.6244127595825468
Bayesian linear regression MSE:  0.3728561316839473
Bayesian linear regression RMSE: 0.6106194655298399


Adding this functionality will allow use to use ensemble methods with other scikit-learn models using the `StackingRegressor`.

---

In [ ]:
model_trace_dict = dict()
for nm in ['k1', 'k2', 'k3', 'k4', 'k5']:
    models_lin[nm].name = 'poly=lin, '+nm
    model_trace_dict.update({models_lin[nm]: traces_lin[nm]})

    models_quad[nm].name = 'poly=quad, '+nm
    model_trace_dict.update({models_quad[nm]: traces_quad[nm]})

In [ ]:
dfwaic = pm.compare(model_trace_dict, ic='WAIC')
dfwaic.index = pd.MultiIndex.from_tuples(
    [tuple(k.split(',')) for k,v in dfwaic.iterrows()])

dfwaic

#### Bayesian linear regression optimisation

- Ridge / lasso regression

### Regression conclusion

- Concluding points on things learnt from dataset.
- Concluding points of Bayesian regression performance.